I played solo for Social Engineering Experts, and we were the only team to full clear the crypto category. Unfortunately it was also the only category we attempted at all. Let's start with the usual imports, and I'll give a high-level description of the idea behind each solve.

In [1]:
from sage.all import *
from Crypto.Util.number import *
from pwn import *

# matrixrsa (508 points / 38 solves)

$(a,x)$ gets encrypted to $(a^e, ea^{e-1}x)$, which equivalently means that $(b,y)$ gets decrypted to $\left(b^{1/e}, \frac{yb^{1/e}}{be}\right)$.

So $(1,1)$ decrypts to $\left(1,\frac{1}{e}\right)$, from which we can learn the modulus $N$ from trial division.

Finally, negation of the second element also carries over in encryption/decryption, so we learn the flag by negating it.

In [2]:
from sage.rings.factorint import factor_trial_division

sh = remote('matrixrsa.chal.crewc.tf', 20001)
sh.readuntil(b'encrypted')
sh.readline()
enc = [int(sh.readline(), 16) for _ in '01']

def getdec(a,b):
    sh.sendline(long_to_bytes(a).hex().encode())
    sh.sendline(long_to_bytes(b).hex().encode())
    sh.readuntil(b'decrypted')
    sh.readline()
    return int(sh.readline(), 16)

foo = getdec(1,1)
e = 65537
N = factor_trial_division(foo*e-1, 2**24)[-1][0]
assert 2047<=N.nbits()<=2048

long_to_bytes(N-getdec(enc[0], N-enc[1]))

[x] Opening connection to matrixrsa.chal.crewc.tf on port 20001
[x] Opening connection to matrixrsa.chal.crewc.tf on port 20001: Trying 34.77.149.230
[+] Opening connection to matrixrsa.chal.crewc.tf on port 20001: Done


b"nY\xfe.\x9f\x17\xb9\xc0\xb6\n\xd9h\xf3L4-\xf4TT&S\x0f\xa9[\x8a\x88\xf6\x8eVr)\xc1A\x82\xfc\x05\xf01\xf4\xb8\x88~\x0c\xf9\xf1\x881\x8a\xcb\xe6\xab\x91F\x7f.\x13\xae\xba>\xf4\x99\xc4\xb1\x8a\x87\x9f\xb7\xa95\xd64}\xb6\xea\x11T\xc9\xa8V\x80\xee\x9f\xe2\xc5{(F>\xcc\x18c[f\x9f+_QB*)z0pW3&\xa59\x03\x86L\xdb\t\x84\xbc\xe57\x0e\xee\xcd\x90\x9b\xeb7\xae\xa3\x07XS\t2\xb2B)#b\xcfZw\xbd!\xde\x83\xaa\x15'q\x99<r\xcbd\xfd\xd3\xeaY\x9dg,,2X\x05 \xba\x89F\xa7\x0e&\xe4\xf3M@\x08v\x1dt\x83a\xffP.\x7f\xd4\x1a\xc0\xf2\x07W\xa0,'\x98\x94\x91\x9bv\xe2v\x91\xf5D\\t\x1e\x1eD\xd6f\xb9\xc8\xbc\xb73\xc9\x96\x8c$\x00crew{h4rd_70_53cur3_f0r_m47r1x_r54}"

# matrixrsa2 (738 points / 28 solves)

$(a,x)$ gets encrypted to $\left(a^e, ea^{e-1}\left(x -a^{\mathrm{0x1337}} - D\right)\right)$, which equivalently means that $(b,y)$ gets decrypted to $\left(b^{1/e}, \frac{yb^{1/e}}{be} + b^{\mathrm{0x1337}/e} + D\right)$.

So $(1,1)$ decrypts to $\left(1,\frac{1}{e} + D + 1\right)$, from which we can learn the modulus $N$ from trial division.

Then $(a^e,0)$ decrypts to $(a,a^{\mathrm{0x1337}} + D)$, and we can learn $a^{\mathrm{0x1337}}$ this way.

Now we know two powers of $a$ that are coprime, so we can solve for $a$ and thus decrypt the flag offline.

In [3]:
sh = remote('matrixrsa2.chal.crewc.tf', 20002)
sh.readuntil(b'encrypted')
sh.readline()
enc = [int(sh.readline(), 16) for _ in '01']

foo = getdec(1,1) - 0xdeadbeef - 1
N = factor_trial_division(foo*e-1, 2**24)[-1][0]
assert 2047<=N.nbits()<=2048

a1337 = getdec(enc[0],0) - 0xdeadbeef
a = mod(a1337, N)**32109 / mod(enc[0], N) ** 2410
long_to_bytes(int(enc[1] / mod(a,N)**(e-1) / e + a1337 + 0xdeadbeef))

[x] Opening connection to matrixrsa2.chal.crewc.tf on port 20002
[x] Opening connection to matrixrsa2.chal.crewc.tf on port 20002: Trying 34.79.210.252
[+] Opening connection to matrixrsa2.chal.crewc.tf on port 20002: Done


b'*\xf7\x95I\xd2p4\x13\x19\xbf\x8dY\xaa\r\xf8[\xc3\x05\x02\xe5\xae\xef\x0e\xc8\x97\xbf\x1e\nk\xd7G\xbf\xa8\x92(\xd5 \xa6\xcewCs\x93g\xce\xa3\xffR\x0c\xa5\xba$hL\x8e\xf4%\xc5\x84\x98\xf9I\x9a\xc5f\xfd\x87N.\xd0\xc1\xbb\x83a\xdd~\x8e\xc0\x14\x7f\x0b\xba\x0b\x10\xe2%\xcd\x0e\x9c[5j1<\xda,\xf1G\x98R\xd7\x9fB\xb6\xef\xdc\x16\x93>\xed\x1c\x0b\xbd-\x9e\x838\x98\x1b\xfaxu\xf4\xb9\xe8\xab\x1f\xd5\xebGh\x05\x96q\x7f\n\xf1?\xb2E\xed,+L\xd8\xfd\xfbZoE\xdbg\xffC\xff\xdek2F|!\xec\xdf\xab\xc1\x85[\xf3\xfc\x96\xfaKv;\x97q\xe9\xed\xe0\x99\x8b\xe7\x96@\xdfx.\xf0\x99\xfcd\x104\xf7\xbbZ\xa8T\x16\xc3Q\xd6\x9c\xdb\x8c|\xa3\xdd\xe6^\x8b,{\x89\n\xd8\xe0)\xbb\x99\xa1\x00crew{4ff1n3_7w34k_15_n07_50_w1ld}'

# frsa (871 points / 20 solves)

The idea here is that $n = \frac{q}{p}$, and $c \equiv m^3 \pmod{n}$.

This means that $c = m^3 + kn = m^3 + k\frac{q}{p}$ for some $k$, which in turn implies $c = \frac{cp}{p} \equiv m^3 \pmod{q}$. Note that $c$ is a rational while $cp$ is an integer.

We can use sagemath's `nearby_rational()` here to automatically convert $n$ and $c$ into $\frac{q}{p}$ and $\frac{cp}{p}$ respectively.

In [4]:
js = {"n": "1.44876550785592503334265950432421420325297582118060042078438550636540910522749678403019518166530642372439366151230264421874410398413833793272618884548160599925817702116098428303450967522096733937366013141930211947792763128151702032735884074911807700800172659563776395192029398799611972719933046988485618648215031959951952004020969378714363476432291148496722805316545151217820262855027336710338345468036062198635525801604111933775316349938234237467504426389267547279109763379541469763502298105176389489292444821569433714782666868004807758394720775374257686871422177558590974495167898763944706851839357352966307465943965426816611269786924465954508226209272577429214625293217383503809862838728273933570714760286039164947492455379435114211698519645274383441843429633685840844671868833406083543006112726092020044873975732743593744382089434443482224495289959971711105417210110744414174520459797753843377179004163098537885284394598652761943331094208978211485724195267631626735387196952551829301994146423483385375867218729187180900985717128576556953248858578571668965475498877692646547265343708306556311150315818083455238289030185403749935603233781292577343183779984671146857543043894776208447081869808627285895849223608318389690026238247512437698041070957028649004541967156188450304649554046232048553105951391797868457722758962995488112908326366838183529376441785620921481022315577057704901497537311757000766882367301276950264333023171883392362048281370037044118644497655423409239667728629840570282957068401983105219619857532874271981787533870146808294248530761364909445403116709201224037169856868078816650521951976062540001056678363829476022845968048056454732435573382891204430645209384281254336926687158471318155195042041104307651433396868633187178055548947656913013706696470202671463098773577117077950748624383953449375499973240182879263325182947154063233656493061790664913628427343370963632626725510930798503142827929687172065476964540376449486231241971970921142388061756878149073011997166705713308138898042308387874199842453364145076872912118034710006839608521650382025036927086417108053258082424017017409392464242881054028353610955750410132084525322691327111336686985498442960600924903226829327798090725381196455391738991420003782010987036147435202882642677112951121901475899707437836500199377431944183023619697352537183409353352318167326258928489538579392384182182849036241624479943746356580732801745018260495447100599071547864140162459313548928342033686638044127324356347559408623154633442462490758902612132055525428393172192329500766724412222181225778012977604996600599282716262237738192784082654926165021783650522079870888957118672899288078040542633396463399277825726466910329073170155707279022862431733940437741757564393035158966495616590325569056880583172014555325930783959090367896737083408965515552749678359672956288246655406506942491012138340228141278314885129425663740940330280512774578942716377906538189668865403917677822363366529879348068579932274366616029428779578368161411323332642557757641290187818290216356837900643653897749361913574593010378542335662672281976082502234645536611500517329162720582302499238228310518711533645095506142546155984056185201394392644692193844707933740577780151992394071443342010784300857612345537439823170419661689370022938536304771304419448503753498015726639720370671278003808768666865485106697733256168473507807659624465947120630126564762373626952385127199872004411320221330748409227649163726500691201159429401400044026576306797022118495260380635214044602593882376411001736210170397746360298199173610170178580647065112280260282987797440324290416909055621283450603411329349781366474728693239312979693620628986752344821113082929830861359911076865162746730666995775818308032737786133254323006032902080799342569176215203450051488375390465979970942783617237304388000440985276384782250849151224833676945184990333181398293372014769606165462793553370051475289382597211019518171517354865761768256815683582718203645523455531264589042699207374820545842905285305058138657243094347062057047809369050234952715087266584576097717230531554578050627080134685826988550369045788589670593141959343427847948821211471704237870681513638937077438242348739635080214653838337244770279414340206825994339667700213229029688599931575494539425146475952923907408592989073017938482821434150251380875370628231626473029462812217242155941425220545647756919469478245063362298483664381589001316155564373314982952361485828048360429071100247956948457623522709933492606591435899499471663221009659335656214510215470951798966083493953769753348042010592188448883960609144891548386527709039199506843925229689933103824004818119970799350266797706850280520077293564848405586628750613494163367880963302787586832628774641142623033281728680193522036332329547281294015690881346978040701658755061720528955391993005111879748905308214089511388311685603846566885066559726409824785011479916172582649195490259272150503392179817918684682100779324201865424206219307395167829276689695193027426626709259572207107902178568198507538378926033150569939891447557989067736538248475724338759869885804865759860692724508247817128937256650744576746110713908946083543608485911151922638581692594146098313829544567711325293325150357144187432338922319660415111104288253252604848048381079044303432151336156247193066049852085492324854332913227879213914019930176256585008075480873197512344165256911155933873167164694902305868999116340219884571875810887319977825221113219779976678840513105202769212326885282268300720460504684031561911506342290315131794444338568865456109459121079012184117743899023789207265745493630293904446958063189775670883895207754576452918415453043677450460895774499962262042489933237180118392194650453640136564290891231905669678468796269079470395729154846594213451531170154625181253884460125717659272063664915078807720617903175659839660430607719344193328885423795746633137798622303411387565564074970665187708132242725716703066711129208927420635340210983968736826682126126568578253268602970977065307418889269076176767438469885915418291889248261329773447059387790062881177342952785076224041060904567197059841159611697460637536884572884994769159940303314801347547465654486615421374057732304977431350432782744183803714749613037663365121373544835038636064551048294975569210817002204589715965296140193810554093107150486919240480809419131393273894168189470814600232080536994999799012709306629194023899814069373106866311624050766092381882654077163764092784397117707039617260348016818524934220643676131344528667517767710357850158012717227640142625095316231807309352339679015351179116056658854058265141031679442203383670816923615955827206833647935911305700741956760637645892969427534976526734917918627600625978148275374533574604870266580708062623987012517327496829841588295068114415656414497328524973240914279210030129862589648051602119901789075011093578447657956189886035334483182486274591838819418100656695024476913545746945596772957999467191144361573398958083176136497623194312434559716496920536148917263811939776317741602573210576116156451674020941025616256467176491937981247273280528940646420300128589715190602368995175193259759064667017806908996991863647894310290370753313205690591574898582396336791522676052598653400959823829059028631584187408135122830010785911711129343559414447094643005811821589914234481832984041310279332297457231739902101889431644872574552807560250452823094089696405021260639180571392713826560837727740888211168982088319360612319983392226764154275732398062403459513304545320204132396101118937004932259684814781276898484948377955674826892841095686039906769051619824507172808819263626712428850177606251770745346131469785463884867882314009687278308275172269003448756459268897261547179288547457075317885663474999225326323668792773507719594966900728172185871118055347667331158300975467936969726168588372459282566816170191529966778816659763943376749254771385124644112789261409664832167603173122062382468938419754422143061449502223368014375232366700861461214775348384456442413476988749917739569931026717968013991772865834107992944477767549845911743879797908084836602876185194407472147600844082474281736494344719976325926883644860415534326624869977297748517216079985854944922002203355180685066554688002489813545840430455436135485998221004881864716843182782346912513897012845126782858572723153488509868764159404007325791372331046989221873631612475744015237636513608438510231990099122816466839729588393958299016686679404303365247691648201703818739673561193844531712845019468832634366835463579393167906777786846209493074638816881833080291470238697071401963976222171685949739046673728718883823666153268442663553768760122601540388119969456569042095489637341911617619853618624739832729784455454457790311132994381206098743453312503929356649988687779561078372372347868308855702930131024310533557412831087330471965815116656746482176564740540911106399530236349698584543556295159740497021097228502911753718245712985419314196594341447513696999455088887929197085176646203516273410954039772664699218799614922391104968411595715208804723587312834594391196887097612155921675830257791121877882011707826350424031381678899530765545564166069128252305041275560540027414797402762303278378810341868305194629471772747180536148986654634781270107946399685452016746373111915155608113109769389128352777505464411042527174157757534139749831786418449133407194334374246720273853964609990172390008189483754856385898606077032429173981159542427152362961152002165529067434480500817051735940031833400796963621322183558073717616411260797547846780056145272119715520751393652979163258161446667042144609159284985073753888567804194826798038246133487565881670478171035760906726561405168533111330911443079356299531042967029423223308085600897284914230010428402339643232847631852525078654067605880368561618224629813593429876620875021863864363727018980093618117746193299049058206620901400894531156315019468104201505591221050632432100470853773306272219008698226690920642316540924190796223091528549794464439401313992125163172546403754282808577063258368693901793484518449862121855221053703922100383714585757518476639832863055303176925990960383170290867501617486393967027775225832786425986414878257491935574056703531944680226059133371700402391293334286636288338645926395002726907574317977298287495757671480038340320926608021378380266056412669493469995272631844057217848752041880471513249116572044771761597711032726829574046465098070139446312231300567666882035407643092269631099948809208678499641628333256686411556105314488254851195767677285156720407304375526551627796586475330367166254732358329314237685679571780783404518680934716379172382493545234195820951711921748135523140262784786291662844947687080368781584002414321132954693821833673586436414283732689610635089954917490071050109663959742080307583554517575758417201267563254536268352805698179195281647180714014272790202347840501528017478491460230881203937337803752904446050860358649224614529162445240603760187888785263478187500551461336353342098514931519303612761842352051901318470582552095183789249101488613562304470986494365354882753671156222386271487105134942990903228553543888257087750703710367681029675272390004173091422471460978074384632213872724595707779333990839019109270752538103770851670234153267318643164186045064511210962727063234520792564052267345782931610896750889981165012394223022274992151088137459816286306621489770789887349484852418097333286932114464793981719733827336798336133706822682584977881785117553719512560127246162255193587755484998604831531589857288468319053230824539759651297374955051813103631105207922037473107406469244526051987378996592516523571489594346958746286785606024287479133954573964248461462547785500293710167251532667290422830351983329299273057377929792600428917999870139061268695540006291688301276033995328265356250987795742711316118604547034159008209247784699001410168502414548295978913404843348857181628156124790220533558832112564116547462210413894676818935060184910102667939158191996221214735338853560587453924754103638407859331954885939323706118856247324681365869473884468168543859663610024542603578023509173891310054684299606490293129312130504161877983544672075276908708637811013478331965751680709459279766880936309939152739703435161282161660660159348777271278573837892971539089346138674715682007045391338964231309318859710039146532167197927762272", "e": "3", "c": "0.0374329037432080346924942741055333040189179298157335893946619400917460815574465091522672435064338691253010187506159909378888439681119439737023903868621433957289033766238516612093113679318176960125153329764184907162335532732883448960274268808106331814418216911000322708991278458264441173073792121532099340363394551733455926115702943267858814223446328856218814352917994684232221852358414526238583050414159808182693538057991816696737543338220502208292660955176068151750043560095938744187194112957604465680221812671646247486877388516540467833106804269177629366417842872443117186335796301368439107696612710982769535319205352783736013116264741057720032008108727256758102523870475532000658838380535264979106915891996709584355853881627779682756416757589034100047223858625328193494177012720072698062768405725375033164023959206184246089271124987995019152998799338091183481981138349753581636837792434066989324342929982026581036417939084475715046706761173451903290335122751469986269564047688640562240554399380016472599231384259218447914097822079363081854981329616599615568345085510900107564467650260108305816893376572741793606341013078627339470913989651448281611762768388961857131798051119082104568508109012595807322667153594675487888342917618388077792179215426546982901573859951153150828839390445536094144293917508617350359450964610729694924323006247389114379410150367339719156961573429775225716954200526452168235510241465294883046893518562668090873786167488813160023116009168028329781494006994811252184516074507348426109530102433158060862111740713892345245493330736058962706313700190573694178886876487252534617521504238273988510654644135906436231661680205586061450088671639899272983328376555100035179445252912488076084636039487805488873511750027062059171452833143017752517060702493412171606768984384703473635616407242590911544668572982096550571301414609143129428301520405244138248607929962490720753318792040536445321556434424053142613762749841781268024321399241483847350656566653329604739621100222895880676162308435667791311693680189784268859673541286825147751252476517936199348309278976311962520826899992605577496920802452245854427897925364430856086866096230640532241031500483879216616114336899178414605483887587282468652020002943282508467114456357063374439236981938325505113155973620457832942056340209720240929972709569857060022618965118938325661512942237753187968711657165980772676885127681004260838946887818349361316210562545134408660383368125116950574878149443671278778081643365672896969547692482287457195182186304973433129534645635272752851942850363709159508710416144813668225996864014921899583886093291989075162191531621026376254721645657205105342348475737505588021305623969815966539025871868634578092363996216884164089986017871188227101268064401279849608061094518462839093329455791459797041752602751437758055210035224971906949084033432229105290146041882301548472801491891136379270092753927972365135231266234000728832967737369263357322446689395777339901259814541310633017508189155354724919148901347834355822625693127423993587303180421150615663786758286351881468150413947903345235605059863327491528701717789183199740763018650651998425626691243854736854538390587592082905996522562985047052987726428636510352779907529360272788520533773746100016497665070894739490204450350255833153172280995398501559089289483144241549923845213496465630634063929993711659401828002200368768883157392760715359221810629838983270725446827531404542936620845055531290009059847281609212589129119821012348204361123794996486315662685282900165344695096415842290957733485231274169728185423514943955020868642001339978691645087270972629965670824869347014190763516029287970291919397386851315717861860903420412301066832081450763733102012764653892395968327800643355216452156491344100755609434214381320334830675473788978807218975351265227016099026111973233530749250201539106006628071405330432437536630928315056928026253185976542195282052861546889067009755832538628219593849186961502624092010089423230665503306704458594406435711398287856379228826754648405000795548624764977984893501496621275851312715864889436586426948466119875823123233317463418402894316132035145917005115825117645608040534418025089169948674259796631407969174564165439427645386647867261419326244581928105699890759740971673073712355258757986513562052909843107792981830863494788474001314749741333988239706176472125053419872302355283907400927516225424303653707203829323201571980071899460227934527086822768275241120488283302852418486990797732118884037566871587044666238260440360014410174297933872217522365823973525838311810304505018613401501583278757260874209169737825788997471361508186027348617510848756940935216883550459840631202113087466048284397386213007804548417434353995195567713380642149010012418182602039063005702770191969676735618417544215755787899896109866180393845699841488554246408349111994549638211994532730331409625929908435778704962188317581671901168458436536169942880396600219330714474588883432980937882365163274365057792910454837986714050320578934888296364909031301059090577938543955401485061833156167303623375249386037812851022763316983883715620414136622589429682534770063255286964501048303829227153318402972717322246722523682656134423318352989494211834916620765629173346163846719836124201302360640590888342040502240898820686398496971182172662689683671002259797468698084526133926072988258061236946525085699450503703644637341759390182348465728086646481762143864897489485389334605085270335867020023665698000741675087326593672919436353490247638128803681382203872203470247054579323331658567569490150515178503516950838202308057873117560642355038907838680756816993805812141139574890081879130496482465205813049116258083365982573938213861373211886617730538030508718754445769952072278244543663469679963716066699155639113713279818706949800631654937347012837903452555192624555447387890938272947298883892659129677515126024887166340088392732242515927394489178739138257341881744729420606103501282632075832914391722801032718091084558286512054204903779171032331493699108887526107271319067506156104929971465554257135037162587555495826603196693479170821356404932847963101638952033061757008210933277233858205066647325423161649017228665170255768550561310674216317198474734964286830850059360071151154153357245451503212103908311259189744805642542106183267245916527780211592261726277380553369710803219286074197091489960375889438986196314518480291290817318515021435558875480316924385661587394202752889660042810413512712552448431548639734217243713133835038950383240809656174255696471080776105543522099545549436269457930238315933300201340843133810338416707050486765505195653655870310293107171800831502147830924015571273826304471466784567628295574470998033782144015787200204860558849751945470898252578636065059969071038562639372408384621993212257126220774734037328771918537249899495208697318139035017186611139552744319101009337834454219096172455279231605505130286369417061267532289205255018634423786690085964717050676260522963106730567712852016865689664748488030367405701273925027572017709934374173339444747117446016092749769321225824880571194841970224237982574760834466714077517141651159426008767094757928405260288550818455643994265766021610711366779218995404379219902597826267530898057927742979232830612666214156360355155632159179894931128509357301434223182927297741851457803596162656614916792345320271770427027878449480007076534501208401382594216288975967641705950999334753494192970507367725161009040220532815819907876806763425851296725166146125907474460741902566420559073665038422279389443815989701429048914791934649884644162063616135534705480159723735356387656117203216797663255575565412619538671503611403375629502269249115741562681865251632500246767757970082861155860613591501611076198780979195226317793306116044004502634614104198271464111313295699030643852184074743464051737856743652340219471002358097156956122005101556725881377239418558073823768741725696111081950987628310534837568784165060803655329328717421066894807391688952226993531249827442453543750373786565753588139068171967261951373928818845683509101814484899725595161110236442822447908095738373857343935195619111884729943496742382155404342181293959358473936496923656429059912716454752036143208503559835143274788911175646664399855964029706021167527829344796345875337526965356238008257009990537518504592000886271905190004273016437951873794575271101906254098965540980602771204753797465579350171498491128348002211133360991420646965464347654128446693664558794809536125314417283953074704464053405291434354484817937310828500052985868644002137448236460442403278097100771598012570969861921757200692492649035891803641403719430116955276858000410807783430574871179213079774303387448359284242555125730933609274137631685093641851755629595878311696188005871895864907011732113490317527855629636783692663391379230352855323951720549309127370564930680599596143091072430182575361828153548318921062249895316059409080758231822941269727898692407181710589036485949767958344312642621828430720661357284054582631654925090104594750149583403183685133488079297250093203713830891734294387306093480306036657039029744911756100994553912042658954140918371483160118005925974097003605564644099421814646985934445959171050906207195159060226709295412449623588843097345905097533900990842590761567969490200517151201201664264424471350684466397663327276382290787031561489073892525306122083197865991451195724602945541782543417113675992553984839423846752724903500518463882558056659749834790385304344374395382438859149813362841545064605124158547571606148971410239545782085845189689922871025985989529199644442198117152581410978564839881988344091289842873330306498348658709109781246730340000144684441081682319972634626339150033978109675430526594092091614032950797859216474588424715713926975297954377470592546377986370390217316502974715276861024799279393633777812065711997962069628920489150044514015959500701841282669686928788423162416509945124526044833045589217195024879034746944395169683432037935348233483160125848842094234057133458052187011004526708827659409682517710699336500726252038604273851028456440169243141248336168915390984883865979748152896392532779996298617722589853657998061702560028593373469128494978928274260196345560813085724083585827107618835501050639897827966470898503034830020424759976545943731818950282617999871767120024748136698988931727022958071546542469268376979736156625973930272793159616043928531517241024287285637390957530534798308928618847034566547990378502838938072554903595064457558680953875024298042003065680812323748404601900047934643644506413091864880945816232963528570042160161997937886355855374798171632766112801633826746938261665574280143030005043732418989149371244127390342608528295712985944291224158550110866195997613231011733044386454983728216749819725108591971257869641796355507611600325312434341888438193612922847184688070544018254780658198693629067895926205153594831258456567172689578181081975419507880976962573758006832026346939881386705707147116072593953060669099963895729354528293128570512925801997665249015256983117485330125186061976560859290776535449625828724129001089644304916728082746246514033264090329837256178831603746853313655427706128539779681498853061015007124905236792213387859518708457369748029182267734651789865723929834269394852605843056519162989901837050017220584431991344285494102025263668759592095658178786009136257303147529063715303800270589505821909946798197392129281218659606383695931020994867660530273169492007606112240557913771536308006680984346519084526917473850341016343793277796962597205535433633706061598501656448716172197207910800186189041112557944930349398116995100032112541878143874553857460370003049883673753665364894265514916804943665560293280654395016239185375080300642949080285886420670277845197829644455340323822848023599928904302039342556656271408171193348287299650207953928208783610441759620713271161462834602010870978182125523909832945660195290562436322515569414410718315576708757545010754808134543695219810999671622504838361812299610877880081994634854571884831828929058968590405505510304189544150207636251480463762738613942826910145088866410221424011471641002902635661841457698522694391175823989186671254805054380705618968778688605044158125976225238286136003770276594948167949417224734456409570720235522471127130990693502835195317681661487948084374481009556056671637258382641007695"}
n,e,c = [RealNumber(x).nearby_rational(max_denominator=2**1000) for x in js.values()]
long_to_bytes(int(mod(c, n.numerator()).nth_root(e)))

b'\xae\xcb3\xae\xd9\xa2j\x15\xb3tGM\xca\xbf\xe8:\xcfJ\x1f\x814Q\x84 \x07\x99\x13\xba/\x08f\xee\xe5)\xd8F\x80\x91\x9cts\x01\x9dc\xbcn\x14\xb3\xe4\x9dxR\x00crew{My_rsa_15_as_vulNEr4b1e_As_plA1nTExt}'

# nec_easy (971 points / 10 solves)

Roughly speaking, we have four equations $nb_i=h+a_id \pmod{r}$ by submitting the same message four times.

Subtracting any two of these gives $\frac{d}{n} = \frac{b_i-b_j}{a_i-a_j}$. We have three of these so can find two different multiples of $q$, gcd them for a small multiple of $q$, then away this small multiple to obtain $q$. This is a factor of $N$, so we are done!

In [5]:
with remote('nec-easy.chal.crewc.tf', 20006) as sh:
    sh.sendlines([b''] * 4)
    def get(z):
        sh.readuntil(z)
        return int(sh.readline(), 16)
    N = get(b'N=')
    e = get(b'e=')
    c = get(b'c=')

    sigs = []
    for _ in range(4):
        sh.readuntil(b'signature: ')
        sigs.append(eval(sh.readline()))

def getnumden(s,t):
    return s[1]-t[1], s[0]-t[0]

n0,d0 = getnumden(sigs[0], sigs[1])
n1,d1 = getnumden(sigs[1], sigs[2])
n2,d2 = getnumden(sigs[2], sigs[3])
q = int(factor(gcd(n0*d1-n1*d0,n1*d2-n2*d1))[-1][0])
print(f'{q=}')
rrr = N // q

d = pow(e, -1, (q-1)*(rrr-1))
long_to_bytes(pow(c, d, N))

[x] Opening connection to nec-easy.chal.crewc.tf on port 20006
[x] Opening connection to nec-easy.chal.crewc.tf on port 20006: Trying 146.148.121.160
[+] Opening connection to nec-easy.chal.crewc.tf on port 20006: Done
[*] Closed connection to nec-easy.chal.crewc.tf port 20006
q=10330468454866735824489915437602166335203003963686127232972164608903480379898085573865678158590796565095279956399988592965064288055048459227227518447242597921040459478959674171344558923671328396012941247822684524315774454657096593811283550313598503244231423959819345469088961


b'\xc7\x8c\xa7\x89"\x82\xbe\x9bx\x0b\x0b\xb9\xbfF\xb9]\x1938\x9d-\xae\xbf\t\xf1q=\xd1\x17\xf0w*\xbe\xecQU\xc1\xa4\xb4\\\n\xb0\'@V$\xbd\xa4\xc7_\xb2\xab7RR.\x9a\x9b\xfd\x1bn\xa3l\xd7+\x1cw\xcc\xf0E\xe7\xa1\x9c\x06\xaa\x16+\x15\xa1\xb0\xb1\xfc\xd8\x8f8\xea.m\xed\xaf\xa7)\xcd\x96a\xe8Z\x89\x95\xa5\x94\xdbc\xc0\xf9 C\xa7Wpv\x9e\xea#\x02^T`\xa8\xb8\xc97XK\xebX4\x98CS\xd8C\x93\x07\xbc\xd3\xec\xd8\xdez\x0e]\x8a!e\xe8\xfb\xfc\\\x80\x1a:\x0el\x9cas\xd2E:\xac\xe3\x169\xd6\x1bf%\xa8\xb8\xc8\x0f]\xf9\xce\x95\xa5\x06\x00crew{mul71pl3_63n3r470r_15_50m371m35_b4d_1d34}'

# pi_is_3.14 (995 points / 5 solves) 🥉

We first note that the least significant bit of XorShift+ is used to determine whether the coordinate is less than 0.5. This is crucial because we can work entirely in $GF(2)$ if we only use the least significant bit.

So what can we do with this? For a starter, if x and y are both less than 0.5, it must be in the circle. Conversely, if we get an "x", at least one `x_lsb` or `y_lsb` is 1. And each of this is just a sum of a subset of 64 bits (i.e. $\mathrm{seed} \in GF(2)^{64}$), e.g. maybe `x[i]_lsb = seed[0] + seed[7] + seed[53]` or something. There are a few ways we can determine the necessary subsets: we could do it symbolically (z3 or sympy), we could use matrices, or I got lazy and just enumerated over the bits, which is slow but whatever.

Finally, it's not entirely linear because `x_lsb==1 or y_lsb==1` is non-linear. By writing it as `(x_lsb+1)(y_lsb+1)==0`, we have quadratic equations, which we can linearise by increasing the number of terms from 64 to 2080. Fortunately, we will have approximately $10000\left(1-\frac{\pi}{4}\right) \approx 2146$ `'x'`s, more than the 2080 we need.

That's about it, most of the script is really bookkeeping, but the maths is basically construct the matrix `xy+x+y` and solve it against the vector of all-ones. The kernel may be non-trivial, but is usually small so we just enumerate through it to find the correct seed.

In [6]:
from pi314 import *
#sh = process(['python', 'pi314.py'])
sh = remote('pi-is-3-14.chal.crewc.tf', 20003)
sh.readline()
line = sh.readline(0)

xys = {i:([],[]) for i in range(10000) if line[i] == ord('x')}

for bp in range(64):
    rand = XorShiftPlus()
    rand.x = 1 << bp if bp < 32 else 0
    rand.y = 1 << (bp-32) if bp >= 32 else 0

    for i in range(10000):
        x = rand.gen32() & 1
        y = rand.gen32() & 1
        
        if i in xys:
            xs, ys = xys[i]
            if x: xs.append(bp)
            if y: ys.append(bp)
            
print(f'{len(xys)=}')

def ind(x,y):
    #return x+y*(y+1)//2 if x<=y else ind(y,x)
    if x == y or y == None:
        return ~x
    return x+y*(y-1)//2 if x<y else ind(y,x)

def makerow(xs,ys):
    v = vector(GF(2), 2080)
    for m in xs + ys:
        v[~m] += 1
    for x in xs:
        for y in ys:
            v[ind(x,y)] += 1
    return v

M = matrix([makerow(*z) for z in xys.values()])
soln0 = M.solve_right([1]*len(xys))

for hom in M.right_kernel():
    soln = soln0 + hom
    rand = XorShiftPlus()
    rand.x = int(''.join(map(str, soln[-32:])), 2)
    rand.y = int(''.join(map(str, soln[-64:-32:])), 2)

    if line == bytes(b'xo'[simulate(rand)] for i in range(10000)):
        break

sh.sendline(bytes(b'xo'[simulate(rand)] for i in range(100)))
sh.readall()

[x] Opening connection to pi-is-3-14.chal.crewc.tf on port 20003
[x] Opening connection to pi-is-3-14.chal.crewc.tf on port 20003: Trying 34.34.177.92
[+] Opening connection to pi-is-3-14.chal.crewc.tf on port 20003: Done
len(xys)=2211


/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3457: DeprecationWarning: solve_right should be called with a vector or matrix
See http://trac.sagemath.org/17405 for details.
  exec(code_obj, self.user_global_ns, self.user_ns)


[x] Receiving all data
[x] Receiving all data: 75B
[x] Receiving all data: 128B
[+] Receiving all data: Done (128B)
[*] Closed connection to pi-is-3-14.chal.crewc.tf port 20003


b'\npi is 3.1156\nBTW, can you predict the following 100 simulation results?\n> crew{3.14159265358979012345678901234567890123456789}\n'

# sspasswd (997 points / 4 solves) 🩸

This was a fun challenge that was reminiscent of my SEETF challenge [🤪onelinecrypto](https://demo.hedgedoc.org/s/DnzmwnCd7). In fact I use that solve script as a base for this one.

So we have an unknown quartic, which is determined from any five points. We have four known points, so guessing any fifth point uniquely determines the polynomial. In particular, we are interested in $f(1)$ and $f(2)$ as these are known to be small (between $0$ and $2^{96}$). There must be a linear relation between these, i.e. $f(2) = mf(1) + c$ for some $m,c$.

Well, this is clearly a lattice problem! We start with $(f(1),f(2)) = (0, c)$ and keep adding multiples of $(1,m)$ (all done mod p). Then we use more lattice stuff to decompose this value into 12 "bytes" each. I say "bytes" because there's nothing in the lattice that forces it between 0 and 256, so it could take large or negative values.

Finally, we use lattice enumeration to find all elements of the lattice where each "byte" here is between 64 and 122 inclusive. It turns out this solution is unique, and we have our flag!

In [7]:
# givens
p = 10455755303881470564335823693050071216876084302779301958643231627211528198918740466518331511682617514785374774155465658746281899684760991037492493121631097
public_shares = [(1337, 1650601573256660700025983099389847404287371792209826894172936673219307987142234073724123985596898136416342832363378752808202043270892758156893643184622917), (48879, 10245506766311536223801309300676898037551029835023534273381238156333461073421971506846811165174309008333508986480294061107462457987212304049599769044657817), (51966, 5859579028489572966139706776167654126678329485615172349824462797003633207598276834487698035757221076083482324371529983809497416849313794719873532398479866), (57005, 1355655372739803925098451066341295601214092302693649405514610829322484191059267898706715750419939661833680243515749318565040749416377065991979895621719206)]

R = GF(p)['x']
a0 = R.lagrange_polynomial(public_shares + [(1, 0)])(x=2)
a1 = R.lagrange_polynomial(public_shares + [(1, 1)])(x=2)

from cpmpy import *

m = matrix(25, 25)
m[0,12] = 1
m[0,0] = a1-a0
m[1,0] = p
for i in range(11):
    m[i+2,i:i+2] = [[-256, 1]]
    m[i+13,i+12:i+14] = [[-256, 1]]
m[-1,0] = a0
m[-1,-1] = isqrt(p) # any large-ish number smaller than p
print(m % 1000)

x = cpm_array(list(intvar(-99999, 99999, 23)) + [1]) @ m.LLL()[:-1,:-1]
Model([x >= 64, x <= 122]).solve()
soln = bytes(list(x.value())[::-1])
print(soln)

long_to_bytes(int(R.lagrange_polynomial(public_shares + [(1, bytes_to_long(soln[:12]))])(x=0)))

[323   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0]
[ 97   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[744   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0 744   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0 744   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0 744   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0 744   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0 744   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0 744   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0 744   1   0   0   0   0   0   0   0   0   0   0   0  

b'crew{ABCD_c4n_u_50lv3_7h15_w17h0u7_1NF0_0f_p45$w0rd_ch4r_7yp35?}'

# nec (999 points / 3 solves) 🥈🧀

See nec_easy above where we discuss how to obtain $r$ (though it was $q$ in nec_easy).

For math reasons I will use $\rho$ to indicate the variable `rrr`.

Now, we know that $q = hr + x$, and $N = q\rho$. A random instance indicates that $(r,q,h,\rho,N,x)$ have bitsizes $(897, 1119, 222, 450, 1568, <458)$.

In particular, we have $N = q\rho \equiv x\rho \pmod{r}$. Specifically $x\rho$ is an integer whose lower "bits" are already known via $N$ (I use "bits" loosely even though it's not a power of 2), and there aren't really many higher bits to determine. So we can use Coppersmith, or in this case it's small enough to brute force.

This gives us $\rho$ as a factor of $N$, and we are done!

In [8]:
with remote('nec.chal.crewc.tf', 20005) as sh:
    sh.sendlines([b''] * 4)
    def get(z):
        sh.readuntil(z)
        return int(sh.readline(), 16)
    N = get(b'N=')
    e = get(b'e=')
    c = get(b'c=')

    sigs = []
    for _ in range(4):
        sh.readuntil(b'signature: ')
        sigs.append(eval(sh.readline()))

def getnumden(s,t):
    return s[1]-t[1], s[0]-t[0]

n0,d0 = getnumden(sigs[0], sigs[1])
n1,d1 = getnumden(sigs[1], sigs[2])
n2,d2 = getnumden(sigs[2], sigs[3])
r = factor(gcd(n0*d1-n1*d0,n1*d2-n2*d1))[-1][0]
print(f'{r=}')

x = Zmod(N)['x'].gen()
f = x*r+N%r
q = int(gcd(N, f(x=f.monic().small_roots(beta=0.1,X=256)[0])))
rrr = N // q

d = pow(e, -1, (q-1)*(rrr-1))
long_to_bytes(pow(c, d, N))

[x] Opening connection to nec.chal.crewc.tf on port 20005
[x] Opening connection to nec.chal.crewc.tf on port 20005: Trying 34.76.226.225
[+] Opening connection to nec.chal.crewc.tf on port 20005: Done
[*] Closed connection to nec.chal.crewc.tf port 20005
r=5035675915009160569761892887162419017865526370791359062395696762449892305293723839777811875888229469887949529427896715490031973313564460584773269213571449657635902401172929221578551034101879820140898142490014983789832042960471975115578939311676613173759137307017893725953


b'\xe6\x94\x978\x05v\xde\xe8\xf5\x7f\x9c\xff\xb8fn\xa8j\xdb \xb21A;\xe9Ic*\xd3\xea\xd8OY\xce+\xce\xb4\xed\xc6\x87W~\x04#\x86I\xc2|\x81s\x1b\x17\t\x92\xb9\x93\xc81g\x8c\x12 \x8d\xa9\xf8\r\xaf\xde\x03r\x90\xf5\xd7\x0c\t\xdd@\xe7\xa6\x8cj\x1c(#E\xd0\x9d_>}\xf5\x93T\xc8\x18\xd3\x0b\xd4\x9e\xc6A\x19\x0cM\xac"o#x\xa5j\x1f\xec\xbd0\xa6\xe5K&\x1b\xa2\xc2?\x04Z\x87[\x9f=\\>\xe0\xddo\x9c\x0b\xac\xdc/\xaf\xe1\xc0\xba\xd0\xac\x00crew{3ll1p71c_curv3_4nd_l4771c3_4r_s0_c0mpl1c473d}'

# rainy (1000 points / 1 solve) 🩸🧀🧀🧀

Maximum cheese! I have no idea what this challenge is about, but there's only 20 keys, so if you connect enough times you will have two instances using the same key. Use one as an oracle for the other.

In [9]:
from hashlib import sha1

dic = {}
while True:
    fl = remote('rainy.chal.crewc.tf', 20004)
    h = sha1(fl.readuntil(b'Wait')).hexdigest()
    print(h)
    if h in dic:
        print('collision!')
        break
    dic[h] = fl

sg = dic[h]
fl.sendline(b'3') #getflag
sg.sendline(b'1') #sign
fl.readuntil(b'sign for msg: ')
sg.send(fl.readline())
sg.readuntil(b'sig: ')
fl.send(sg.readline())
fl.readall()

[x] Opening connection to rainy.chal.crewc.tf on port 20004
[x] Opening connection to rainy.chal.crewc.tf on port 20004: Trying 34.77.23.235
[+] Opening connection to rainy.chal.crewc.tf on port 20004: Done
a80be496719a8302aca195e6d34e6f27c8fbfc7f
[x] Opening connection to rainy.chal.crewc.tf on port 20004
[x] Opening connection to rainy.chal.crewc.tf on port 20004: Trying 34.77.23.235
[+] Opening connection to rainy.chal.crewc.tf on port 20004: Done
0d0561d7f0d671ea6e745d35475a8019fdf7bf25
[x] Opening connection to rainy.chal.crewc.tf on port 20004
[x] Opening connection to rainy.chal.crewc.tf on port 20004: Trying 34.77.23.235
[+] Opening connection to rainy.chal.crewc.tf on port 20004: Done
d92182cdc5231465a0b28ce99e4c14663fa8cb69
[x] Opening connection to rainy.chal.crewc.tf on port 20004
[x] Opening connection to rainy.chal.crewc.tf on port 20004: Trying 34.77.23.235
[+] Opening connection to rainy.chal.crewc.tf on port 20004: Done
1c27debf8a88da14ebb23fb1140ef280cef23747
[x] Open

b'sig(hex): Congratulations. Here is FLAG.\ncrew{Dr34m5_c0m3_7ru3_0v3r_7h3_r41nb0w!}\n'